In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import requests
import json
import pandas as pd

In [ ]:

df3=pd.read_csv("../multiclass_dataset/test.csv")

In [ ]:
df3['Label'] = df3['Label'].replace({0: 'Non-sensitive', 1: 'Secret'})
print(df3['Label'].value_counts())

In [ ]:

df3['Label'] = df3['Label'].replace({0: 'Non-sensitive', 1: 'Secret'})
print(df3['Label'].value_counts())

In [ ]:
def create_context_window(text, target_string, window_size=200):

    target_index = text.find(target_string)

    if target_index != -1:
        start_index = max(0, target_index - window_size)
        end_index = min(len(text), target_index + len(target_string) + window_size)
        context_window = text[start_index:end_index]
        return context_window

    return None


df3['Contents'] = df3.apply(lambda row: create_context_window(row['Contents'], row['Secret']), axis=1)

In [ ]:

X_test = df3

In [ ]:

def generate_test_prompt(data_point):
    return f"""
            Classify the given candidate string into one of the following categories based on its presence and usage in the provided code snippet:
            
            - Private Key  
            - API Key and Secret  
            - Authentication Key and Token  
            - Other  
            - Generic Secret  
            - Database and Server URL  
            - Password  
            - Username  

            A secret refers to sensitive information like API keys, passwords, private tokens, and credentials. Analyze the candidate string in the given code snippet and determine its correct category. Only output the category name exactly as given—no explanations or additional text .

candidate_string: {data_point["Secret"]}
code snippet: {data_point["Contents"]}
label: """.strip()

In [ ]:

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'Category']
X_test['prompt'] = X_test.apply(generate_test_prompt, axis=1)

In [ ]:
OPENAI_API_URL = "https://api.openai.com/v1/chat/completions"
API_KEY = "YOUR_API_KEY_HERE"  # Replace with your actual API key

In [ ]:

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

def query_gpt4o(prompt):
    payload = {
        "model": "gpt-4o",  # using GPT-4o
        "messages": [
            {
                "role": "user",
                "content": prompt
            },
            {
                "role": "system",
                "content": "You are a code security auditor or classifier speccialized in identifying and categorizing sensitive secrets from code snippet."
            }
        ],
        "temperature": 0,
        "max_tokens": 5  # few extra tokens in case it says 'Non-sensitive' etc
    }

    response = requests.post(OPENAI_API_URL, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content'].strip()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Perform zero-shot classification
from tqdm import tqdm
import time

y_pred = []

for prompt in tqdm(X_test['prompt'], desc="Processing Prompts"):
    success = False
    retries = 3
    response = None

    while not success and retries > 0:
        response = query_gpt4o(prompt)
        if response:
            success = True
        else:
            retries -= 1
            time.sleep(1.5)  # wait before retrying

    if success:
        response = response.replace('"', '').replace("'", "").strip()
        
        if "Private Key" in response:
            y_pred.append("Private Key")
        elif "API Key and Secret" in response:
            y_pred.append("API Key and Secret")
        elif "Authentication Key and Token" in response:
            y_pred.append("Authentication Key and Token")
        elif "Generic Secret" in response:
            y_pred.append("Generic Secret")
        elif "Database and Server URL" in response:
            y_pred.append("Database and Server URL")
        elif "Password" in response:
            y_pred.append("Password")
        elif "Username" in response:
            y_pred.append("Username")
        else:
            print(f"Unexpected response: {response}")
            y_pred.append("Other")  # fallback for unknown label
    else:
        y_pred.append("Other")  # fallback if all retries fail

    # Slight delay to avoid hitting the token-per-minute rate limit
    time.sleep(0.7)






In [ ]:
# prompt = "What is your exact model version? Are you DeepSeek-V1, V2, V3, or R1?"
# response = requests.post(
#     "https://api.deepseek.com/v1/chat/completions",
#     headers={"Authorization": f"Bearer {API_KEY}"},
#     json={
#         "model": "deepseek-chat",
#         "messages": [{"role": "user", "content": prompt}],
#     },
# )
# print(response.json()["choices"][0]["message"]["content"])  

In [ ]:
def evaluate(y_true, y_pred):
    # Define category mappings
    categories = [
        "Private Key",
        "API Key and Secret",
        "Authentication Key and Token",
        "Other",
        "Generic Secret",
        "Database and Server URL",
        "Password",
        "Username"
    ]
    category_map = {category: i for i, category in enumerate(categories)}

    # Map string labels to integer values
    y_true_mapped = np.array([category_map[label] for label in y_true])
    y_pred_mapped = np.array([category_map[label] for label in y_pred])

    # Calculate overall accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Overall Accuracy: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(
        y_true=y_true_mapped, 
        y_pred=y_pred_mapped, 
        target_names=categories, 
        digits=4
    )
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(
        y_true=y_true_mapped, 
        y_pred=y_pred_mapped
    )
    print('\nConfusion Matrix:')
    print(conf_matrix)

evaluate(y_true, y_pred)



In [ ]:

def plot_confusion_matrix(y_true, y_pred):
    # Define category mappings
    categories = [
        "Private Key",
        "API Key and Secret",
        "Authentication Key and Token",
        "Other",
        "Generic Secret",
        "Database and Server URL",
        "Password",
        "Username"
    ]
    category_map = {category: i for i, category in enumerate(categories)}

    # Map string labels to integer values
    y_true_mapped = np.array([category_map[label] for label in y_true])
    y_pred_mapped = np.array([category_map[label] for label in y_pred])

    # Compute confusion matrix
    conf_matrix = confusion_matrix(y_true_mapped, y_pred_mapped)

     # Generate confusion matrix heatmap with correct labels
    filename = '../plots/deepseek-v3-multi.png'
    base_filename = os.path.splitext(os.path.basename(filename))[0]

    # Plot the confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=categories, yticklabels=categories)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Confusion Matrix: {base_filename}")
    plt.savefig(filename, bbox_inches='tight')
    plt.show()

# Call the function to plot the confusion matrix
plot_confusion_matrix(y_true, y_pred)
